# Random Forest Classifier

A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is controlled with the max_samples parameter if bootstrap=True (default), otherwise the whole dataset is used to build each tree.

Source: [Random Forest Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=random%20forest#sklearn.ensemble.RandomForestClassifier)


In [1]:
# Update sklearn to prevent version mismatches
# -------------------------------------------
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\alysh\anaconda3\envs\pythondata\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing
# -------------------------------------------
!pip install joblib

In [3]:
# Dependencies and Setup
# -------------------------------------------
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Read exploplanet csv data file
# -------------------------------------------
df = pd.read_csv("../data/exoplanet_data.csv")
# Drop the null columns where all values are null
# -------------------------------------------
df = df.dropna(axis='columns', how='all')
# Drop the null rows
# -------------------------------------------
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


## Select your features (columns)

In [5]:
# Use `koi_disposition` for the y values
# -------------------------------------------
y = df["koi_disposition"]
y

0            CONFIRMED
1       FALSE POSITIVE
2       FALSE POSITIVE
3            CONFIRMED
4            CONFIRMED
             ...      
6986    FALSE POSITIVE
6987    FALSE POSITIVE
6988         CANDIDATE
6989    FALSE POSITIVE
6990    FALSE POSITIVE
Name: koi_disposition, Length: 6991, dtype: object

In [6]:
# Assign X df and drop "koi_disposition"
# -------------------------------------------
X = df.drop(columns=["koi_disposition"])
X.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [7]:
print(df.columns)

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')


In [8]:
# Set features. This will also be used as your x values.
# selected_features = df[['names', 'of', 'selected', 'features', 'here']]

## Create a Train Test Split

In [9]:
# Split our data into training and testing
# -------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
3875,1,0,0,0,11.377528,0.000111,-0.000111,132.58480,0.00833,-0.00833,...,-305,4.162,0.128,-0.192,1.660,0.528,-0.352,300.19409,45.145741,13.930
1768,0,0,0,0,2.215713,0.000008,-0.000008,131.82107,0.00298,-0.00298,...,-169,4.447,0.108,-0.162,0.901,0.195,-0.130,297.02008,43.432549,15.392
3250,0,0,0,0,7.785911,0.000203,-0.000203,137.87340,0.02790,-0.02790,...,-180,4.479,0.054,-0.216,0.954,0.305,-0.102,290.42307,51.388729,13.515
6574,1,0,1,1,2.404557,0.000004,-0.000004,131.67616,0.00144,-0.00144,...,-175,4.013,0.259,-0.130,1.775,0.411,-0.503,293.21356,46.175129,13.474
2815,0,0,0,0,110.461746,0.002985,-0.002985,147.54650,0.02380,-0.02380,...,-206,4.434,0.054,-0.216,1.058,0.349,-0.116,287.03952,46.481701,15.092


## Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
# Scale your data
# -------------------------------------------
# Create a StandardScater model and fit it to the training data
# -------------------------------------------
X_scaler = StandardScaler().fit(X_train)

# Transform the training and testing data using the X_scaler and y_scaler models
# -------------------------------------------
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Train the Model

In [11]:
# Create a Random Forest Model
# -------------------------------------------
classifier = RandomForestClassifier(n_estimators=200)
classifier

RandomForestClassifier(n_estimators=200)

In [12]:
# Fit (train) or model using the training data
# -------------------------------------------
classifier.fit(X_train_scaled, y_train)

RandomForestClassifier(n_estimators=200)

In [13]:
# Validate the model using the test data
# -------------------------------------------
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.8984989278055754


## Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [14]:
# LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
#           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
#           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
#           verbose=0, warm_start=False)

In [15]:
# Create the GridSearchCV model
# -------------------------------------------
param_grid = {'criterion': ["gini", "entropy"]}
grid = GridSearchCV(classifier, param_grid, verbose=3)

In [16]:
# Train the model with GridSearch
# -------------------------------------------
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END ....................criterion=gini;, score=0.900 total time=   2.8s
[CV 2/5] END ....................criterion=gini;, score=0.909 total time=   2.7s
[CV 3/5] END ....................criterion=gini;, score=0.890 total time=   2.8s
[CV 4/5] END ....................criterion=gini;, score=0.885 total time=   2.7s
[CV 5/5] END ....................criterion=gini;, score=0.882 total time=   2.8s
[CV 1/5] END .................criterion=entropy;, score=0.898 total time=   4.2s
[CV 2/5] END .................criterion=entropy;, score=0.903 total time=   4.1s
[CV 3/5] END .................criterion=entropy;, score=0.893 total time=   4.2s
[CV 4/5] END .................criterion=entropy;, score=0.881 total time=   4.1s
[CV 5/5] END .................criterion=entropy;, score=0.883 total time=   4.1s


GridSearchCV(estimator=RandomForestClassifier(n_estimators=200),
             param_grid={'criterion': ['gini', 'entropy']}, verbose=3)

In [17]:
print(grid.best_params_)
print(grid.best_score_)

{'criterion': 'gini'}
0.8930574672952627


# Save the Model

In [18]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
# -------------------------------------------
import joblib
filename = '../models/model_3_RF.sav'
joblib.dump(grid, filename)

['../models/model_3_RF.sav']

## Random Forest Classifier Results

In [19]:
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")
print(f"Best Parameters: {(grid.best_params_)}")
print(f"Best GRid Score: {(grid.best_score_)}")

Training Data Score: 1.0
Testing Data Score: 0.8984989278055754
Best Parameters: {'criterion': 'gini'}
Best GRid Score: 0.8930574672952627
